### Converting new bookings forecasts for deferred revenue model

Since Karen Burgess changed roles, I now have two sources of data for the bookings forecast

DX - 
DME - 

The data files they provide are very different and I need to change them to get one bookings dataframe


In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

### DME bookings

In [2]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DME_Bookings_FY21_Plan.xlsx'
sheetname = 'Raw'
df_DME = pd.read_excel(filename, sheetname)

In [3]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4403 entries, 0 to 4402
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Metrics          4403 non-null   object 
 1   Profit center    4403 non-null   object 
 2   Market Area      4403 non-null   object 
 3   Market Segement  4403 non-null   object 
 4   GTM              4403 non-null   object 
 5   Q1 2021          4403 non-null   float64
 6   Q2 2021          4403 non-null   float64
 7   Q3 2021          4403 non-null   float64
 8   Q4 2021          4403 non-null   float64
 9   2021             4403 non-null   float64
dtypes: float64(5), object(5)
memory usage: 344.1+ KB


In [4]:
df_DME.sample(5)

,Metrics,Profit center,Market Area,Market Segement,GTM,Q1 2021,Q2 2021,Q3 2021,Q4 2021,2021
512,Total Subscription Attrition,13450 - Stock Photography,North America (R),EDUCATION,Corporate,-4717.187492,-1180.052328,-14050.113959,-3571.672458,-23519.026237
991,Total Subscription Attrition,IS17 - Adobe Sign,China (MA),COMMERCIAL,Strategic,0.000000,0.000000,-1155.000000,0.000000,-1155.000000
468,Total Subscription Attrition,IS18 - Stock Photography,Southeast Asia (MA),COMMERCIAL,Strategic,-3991.471458,0.000000,0.000000,-1113.277647,-5104.749106
2826,Net ACV,14400 - Adobe Sign,Aus and New Zealand (MA),EDUCATION,Strategic,40438.164055,49111.989655,46913.364055,-48312.503745,88151.014020
3775,ASV & Usage Based,EB15 - Document Cloud,Iberica (MA),GOVERNMENT,Corporate,30228.483301,52605.146836,50524.283577,77021.010438,210378.924152


In [5]:
# Clean Column Names
df_DME.columns

Index(['Metrics', 'Profit center', 'Market Area', 'Market Segement', 'GTM',
       'Q1 2021', 'Q2 2021', 'Q3 2021', 'Q4 2021', '2021'],
      dtype='object')

### The DX bookings have no information about segment. We need to delete segment and then group by to following
- pc_descr
- geo
- region
- market_area


In [6]:
df_DME = df_DME.rename(columns = {'Metrics': 'metrics',
                        'Profit center': 'profit_center',
                        'Market Area': 'market_area',
                        'Market Segement': 'segment',
                        'Q1 2021':'Q1_2021',
                        'Q2 2021':'Q2_2021',
                        'Q3 2021':'Q3_2021',
                        'Q4 2021':'Q4_2021' 
                        })

In [7]:
df_DME = df_DME.drop(columns = ['segment', 'GTM', '2021'])

In [8]:
df_DME.columns

Index(['metrics', 'profit_center', 'market_area', 'Q1_2021', 'Q2_2021',
       'Q3_2021', 'Q4_2021'],
      dtype='object')

### Clear out anything that is not 'NET ASV'


In [9]:
df_DME['metrics'].value_counts(dropna=False)

Net ACV                         1797
ASV & Usage Based               1510
Total Subscription Attrition    1096
Name: metrics, dtype: int64

In [10]:
df_DME = df_DME[df_DME['metrics']=='Net ACV']

In [11]:
df_DME.head(5)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
1096,Net ACV,EB10 - Creative,AMER (G),1.283097e+07,1.326238e+07,1.943717e+07,2.068369e+07
1097,Net ACV,EB10 - Creative,AMER (G),-4.202836e+05,8.299171e+05,1.133272e+06,1.270177e+06
1098,Net ACV,EB10 - Creative,AMER (G),3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06
1099,Net ACV,EB10 - Creative,AMER (G),-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05
1100,Net ACV,EB10 - Creative,AMER (G),2.279720e+06,2.284183e+06,3.444284e+06,3.827079e+06


# THE DME profit_center is completely nested on the DME raw sheet.

We will need to understand the nesting and see if there is a way to adjust for this.


In [12]:
df_DME['profit_center'].value_counts()

GP10 - Creative                   161
IS10 - Creative - Professional    161
10100 - Design                    161
EB10 - Creative                   161
EB15 - Document Cloud             155
GP15 - Document Cloud             155
10800 - Acrobat Desk              146
IS15 - Acrobat DC                 146
14400 - Adobe Sign                102
IS17 - Adobe Sign                 102
IS18 - Stock Photography           97
13450 - Stock Photography          97
10850 - DCE                        78
10110 - CCE + Stock                75
Name: profit_center, dtype: int64

# After checking the spreadsheet, it appears GP10 numbers map to EB10 and EB15 numbers map to GP15.
So there is no further subset of the broader EB mapping to GP (not a seperate GP within an EB)

### To fix  the data here, we will split the profit center based on the -.
- First column is BU_id
- second column is BU

We will then filter the dataframe for including only EB10 and EB15.
The BU will be kept (Creative, Document Cloud)


In [13]:
# creating the BU_ID
df_DME['BU_id'] =  df_DME['profit_center'].apply(lambda st: st[0:st.find("-")])
df_DME['BU_segment'] = df_DME['profit_center'].apply(lambda st: st[st.find("-")+1:])

In [14]:
df_DME['BU_id'] = df_DME['BU_id'].str.strip()
df_DME['BU_segment'] = df_DME['BU_segment'].str.strip()

In [15]:
# check the BU_id value counts
df_DME['BU_id'].value_counts(dropna=False)

10100    161
IS10     161
GP10     161
EB10     161
EB15     155
GP15     155
10800    146
IS15     146
14400    102
IS17     102
IS18      97
13450     97
10850     78
10110     75
Name: BU_id, dtype: int64

In [16]:
list_BU_keepers = ['EB10', 'EB15']

In [17]:
df_DME = df_DME[df_DME['BU_id'].isin(list_BU_keepers)]

In [18]:
df_DME['BU_segment'].value_counts(dropna=False)

Creative          161
Document Cloud    155
Name: BU_segment, dtype: int64

## Now Dealing with the GEO, Region, Market Area mess

It looks like there are three seperate cuts of the data that are a hierarchy

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [19]:
# identify the characters in a string
df_DME['in_parens'] =  df_DME['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])

In [20]:
df_DME['in_parens'].value_counts()

MA    177
R     102
G      37
Name: in_parens, dtype: int64

In [21]:
# now delete the parenthesis and the items between the parenthesis in the 'Market Area' column
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316 entries, 1096 to 2163
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   metrics        316 non-null    object 
 1   profit_center  316 non-null    object 
 2   market_area    316 non-null    object 
 3   Q1_2021        316 non-null    float64
 4   Q2_2021        316 non-null    float64
 5   Q3_2021        316 non-null    float64
 6   Q4_2021        316 non-null    float64
 7   BU_id          316 non-null    object 
 8   BU_segment     316 non-null    object 
 9   in_parens      316 non-null    object 
dtypes: float64(4), object(6)
memory usage: 27.2+ KB


In [22]:
df_DME['market_area'] = df_DME['market_area'].apply(lambda st: st[0:st.find("(")-1])
df_DME.head(5)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens
1096,Net ACV,EB10 - Creative,AMER,1.283097e+07,1.326238e+07,1.943717e+07,2.068369e+07,EB10,Creative,G
1097,Net ACV,EB10 - Creative,AMER,-4.202836e+05,8.299171e+05,1.133272e+06,1.270177e+06,EB10,Creative,G
1098,Net ACV,EB10 - Creative,AMER,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,G
1099,Net ACV,EB10 - Creative,AMER,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,EB10,Creative,G
1100,Net ACV,EB10 - Creative,AMER,2.279720e+06,2.284183e+06,3.444284e+06,3.827079e+06,EB10,Creative,G


In [23]:
df_DME['market_area'].value_counts()

Korea                   24
India                   18
United States           15
AMER                    15
North America           15
ASIA                    12
China                   12
Southeast Asia          12
SEA                     12
ANZ                     12
Greater China           12
Aus and New Zealand     12
Canada                  10
Brazil                   8
Latin America            8
Japan                    8
Hong Kong & Taiwan       8
Germany                  6
United Kingdom           6
Benelux                  6
Nordic                   6
Middle East              6
Iberica                  6
Southwest Europe         6
Central Europe           6
Strat. Latin America     6
Northern Europe          6
Italy                    6
France                   6
Switzerland              6
EMEA                     6
SSA & Israel             6
JPN                      4
Russia & CIS             3
Eastern Europe           3
Mexico                   3
Name: market_area, dtype: in

In [24]:
df_DME.head(10)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens
1096,Net ACV,EB10 - Creative,AMER,1.283097e+07,1.326238e+07,1.943717e+07,2.068369e+07,EB10,Creative,G
1097,Net ACV,EB10 - Creative,AMER,-4.202836e+05,8.299171e+05,1.133272e+06,1.270177e+06,EB10,Creative,G
1098,Net ACV,EB10 - Creative,AMER,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,G
1099,Net ACV,EB10 - Creative,AMER,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,EB10,Creative,G
1100,Net ACV,EB10 - Creative,AMER,2.279720e+06,2.284183e+06,3.444284e+06,3.827079e+06,EB10,Creative,G
1101,Net ACV,EB10 - Creative,AMER,1.520516e+06,1.897964e+06,1.755988e+06,1.382500e+06,EB10,Creative,G
1102,Net ACV,EB10 - Creative,AMER,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,EB10,Creative,G
1103,Net ACV,EB10 - Creative,North America,1.163249e+07,1.207200e+07,1.763207e+07,1.866415e+07,EB10,Creative,R
1104,Net ACV,EB10 - Creative,North America,-3.647437e+05,8.942209e+05,1.166460e+06,1.287489e+06,EB10,Creative,R
1105,Net ACV,EB10 - Creative,North America,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,R


In [25]:
df_DME['pc_ID'] = df_DME['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DME['pc_descr'] = df_DME['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [26]:
df_DME.sample(20)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens,pc_ID,pc_descr
2019,Net ACV,EB15 - Document Cloud,North America,8.847759e+05,1.226003e+06,1.895016e+06,2.396398e+06,EB15,Document Cloud,R,EB15,Document Cloud
2093,Net ACV,EB15 - Document Cloud,Korea,5.625000e+04,7.500000e+04,9.375000e+04,1.125000e+05,EB15,Document Cloud,R,EB15,Document Cloud
2012,Net ACV,EB15 - Document Cloud,AMER,7.746883e+05,1.028167e+06,1.574853e+06,2.052226e+06,EB15,Document Cloud,G,EB15,Document Cloud
2088,Net ACV,EB15 - Document Cloud,India,-1.642432e+03,6.204070e+03,5.108075e+03,3.388035e+03,EB15,Document Cloud,MA,EB15,Document Cloud
2159,Net ACV,EB15 - Document Cloud,JPN,1.801736e+06,2.437668e+06,2.164097e+06,2.676546e+06,EB15,Document Cloud,G,EB15,Document Cloud
2110,Net ACV,EB15 - Document Cloud,Southeast Asia,1.010013e+04,1.301613e+04,1.261597e+04,1.673413e+04,EB15,Document Cloud,MA,EB15,Document Cloud
2138,Net ACV,EB15 - Document Cloud,SSA & Israel,0.000000e+00,-1.284426e+04,0.000000e+00,0.000000e+00,EB15,Document Cloud,MA,EB15,Document Cloud
2083,Net ACV,EB15 - Document Cloud,India,2.296258e+05,2.939707e+05,3.203420e+05,3.187668e+05,EB15,Document Cloud,R,EB15,Document Cloud
1122,Net ACV,EB10 - Creative,Latin America,1.198482e+06,1.190388e+06,1.805103e+06,2.019533e+06,EB10,Creative,R,EB10,Creative
2147,Net ACV,EB15 - Document Cloud,Benelux,0.000000e+00,4.934869e+03,0.000000e+00,8.629193e+03,EB15,Document Cloud,MA,EB15,Document Cloud


adding columns geo, region, market_area (which already exists)

In [27]:
df_DME['geo'] = df_DME[df_DME['in_parens']=='G']['market_area']

In [28]:
df_DME.tail(60)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens,pc_ID,pc_descr,geo
2104,Net ACV,EB15 - Document Cloud,SEA,1.010013e+04,1.301613e+04,1.261597e+04,1.673413e+04,EB15,Document Cloud,R,EB15,Document Cloud,NaN
2105,Net ACV,EB15 - Document Cloud,SEA,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,EB15,Document Cloud,R,EB15,Document Cloud,NaN
2106,Net ACV,EB15 - Document Cloud,SEA,1.296345e+04,3.396644e+04,1.981394e+05,2.928341e+05,EB15,Document Cloud,R,EB15,Document Cloud,NaN
2107,Net ACV,EB15 - Document Cloud,SEA,3.900000e+04,4.875000e+04,4.875000e+04,6.500000e+04,EB15,Document Cloud,R,EB15,Document Cloud,NaN
2108,Net ACV,EB15 - Document Cloud,Southeast Asia,4.620200e+05,4.304577e+05,3.570979e+05,4.613317e+05,EB15,Document Cloud,MA,EB15,Document Cloud,NaN
2109,Net ACV,EB15 - Document Cloud,Southeast Asia,2.098108e+05,2.503129e+05,2.777980e+05,2.953658e+05,EB15,Document Cloud,MA,EB15,Document Cloud,NaN
2110,Net ACV,EB15 - Document Cloud,Southeast Asia,1.010013e+04,1.301613e+04,1.261597e+04,1.673413e+04,EB15,Document Cloud,MA,EB15,Document Cloud,NaN
2111,Net ACV,EB15 - Document Cloud,Southeast Asia,5.578341e+03,5.578341e+03,5.578341e+03,1.007834e+04,EB15,Document Cloud,MA,EB15,Document Cloud,NaN
2112,Net ACV,EB15 - Document Cloud,Southeast Asia,1.296345e+04,3.396644e+04,1.981394e+05,2.928341e+05,EB15,Document Cloud,MA,EB15,Document Cloud,NaN
2113,Net ACV,EB15 - Document Cloud,Southeast Asia,3.900000e+04,4.875000e+04,4.875000e+04,6.500000e+04,EB15,Document Cloud,MA,EB15,Document Cloud,NaN


In [29]:
df_DME['geo'] =df_DME['geo'].ffill()

In [30]:
df_DME['geo'].value_counts(dropna=False)

ASIA    134
EMEA     90
AMER     80
JPN      12
Name: geo, dtype: int64

In [31]:
#df_DME.head(60)

### Comfortable that the forward fill of 'geo' worked well

### Working on the region now
Note: There will be a problem with the geo values here. They will all contain a region even if they are subtotals. This is OK because we will only be using the market area data and can recreate the region and geo data thanks to the new columns we have created


In [32]:
df_DME['region'] = df_DME[df_DME['in_parens']=='R']['market_area']
df_DME.head(20)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens,pc_ID,pc_descr,geo,region
1096,Net ACV,EB10 - Creative,AMER,1.283097e+07,1.326238e+07,1.943717e+07,2.068369e+07,EB10,Creative,G,EB10,Creative,AMER,NaN
1097,Net ACV,EB10 - Creative,AMER,-4.202836e+05,8.299171e+05,1.133272e+06,1.270177e+06,EB10,Creative,G,EB10,Creative,AMER,NaN
1098,Net ACV,EB10 - Creative,AMER,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,G,EB10,Creative,AMER,NaN
1099,Net ACV,EB10 - Creative,AMER,-2.594523e+05,-3.009332e+05,-1.303246e+06,-4.993909e+05,EB10,Creative,G,EB10,Creative,AMER,NaN
1100,Net ACV,EB10 - Creative,AMER,2.279720e+06,2.284183e+06,3.444284e+06,3.827079e+06,EB10,Creative,G,EB10,Creative,AMER,NaN
1101,Net ACV,EB10 - Creative,AMER,1.520516e+06,1.897964e+06,1.755988e+06,1.382500e+06,EB10,Creative,G,EB10,Creative,AMER,NaN
1102,Net ACV,EB10 - Creative,AMER,-2.833680e+03,0.000000e+00,-1.859105e+04,-3.333574e+03,EB10,Creative,G,EB10,Creative,AMER,NaN
1103,Net ACV,EB10 - Creative,North America,1.163249e+07,1.207200e+07,1.763207e+07,1.866415e+07,EB10,Creative,R,EB10,Creative,AMER,North America
1104,Net ACV,EB10 - Creative,North America,-3.647437e+05,8.942209e+05,1.166460e+06,1.287489e+06,EB10,Creative,R,EB10,Creative,AMER,North America
1105,Net ACV,EB10 - Creative,North America,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,R,EB10,Creative,AMER,North America


In [33]:
df_DME['region'] = df_DME['region'].ffill()
df_DME['region'].value_counts(dropna=False)

North America       40
Latin America       37
Southwest Europe    34
Greater China       32
Northern Europe     30
SEA                 30
ANZ                 24
Central Europe      24
Korea               24
India               18
Japan               16
NaN                  7
Name: region, dtype: int64

In [34]:
df_DME.tail(40)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens,pc_ID,pc_descr,geo,region
2124,Net ACV,EB15 - Document Cloud,Switzerland,0.000000e+00,1.942695e+02,3.872107e+04,1.439838e+04,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Central Europe
2125,Net ACV,EB15 - Document Cloud,Switzerland,7.161297e+04,6.614120e+04,9.519540e+04,3.261996e+05,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Central Europe
2126,Net ACV,EB15 - Document Cloud,Eastern Europe,3.363213e+04,3.619620e+04,1.591253e+05,2.045383e+05,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Central Europe
2127,Net ACV,EB15 - Document Cloud,Russia & CIS,3.529556e+04,4.880611e+04,5.010099e+04,1.160239e+05,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Central Europe
2128,Net ACV,EB15 - Document Cloud,Northern Europe,3.764673e+06,4.523071e+06,3.921933e+06,8.809946e+06,EB15,Document Cloud,R,EB15,Document Cloud,EMEA,Northern Europe
2129,Net ACV,EB15 - Document Cloud,Northern Europe,-4.310175e+03,-1.284426e+04,5.887947e+01,4.747280e+03,EB15,Document Cloud,R,EB15,Document Cloud,EMEA,Northern Europe
2130,Net ACV,EB15 - Document Cloud,Northern Europe,1.687353e+05,5.869572e+05,4.832212e+05,6.640627e+05,EB15,Document Cloud,R,EB15,Document Cloud,EMEA,Northern Europe
2131,Net ACV,EB15 - Document Cloud,Nordic,9.290953e+05,1.152535e+06,8.365626e+05,2.779185e+06,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Northern Europe
2132,Net ACV,EB15 - Document Cloud,Nordic,0.000000e+00,0.000000e+00,0.000000e+00,1.948999e+03,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Northern Europe
2133,Net ACV,EB15 - Document Cloud,Nordic,2.610646e+04,6.402820e+04,1.143170e+05,1.823834e+05,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Northern Europe


#### Now we only want to keep the market areas

In [35]:
df_DME = df_DME[df_DME['in_parens']=='MA'].copy()
df_DME['market_area'].value_counts(dropna=False)

United States           15
Aus and New Zealand     12
Korea                   12
China                   12
Southeast Asia          12
Canada                  10
India                    9
Brazil                   8
Hong Kong & Taiwan       8
France                   6
United Kingdom           6
Benelux                  6
Nordic                   6
Middle East              6
Switzerland              6
Iberica                  6
Germany                  6
Italy                    6
Strat. Latin America     6
SSA & Israel             6
Japan                    4
Russia & CIS             3
Mexico                   3
Eastern Europe           3
Name: market_area, dtype: int64

In [36]:
len(df_DME)

177

In [37]:
df_DME.sample(20)

,metrics,profit_center,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,in_parens,pc_ID,pc_descr,geo,region
1117,Net ACV,EB10 - Creative,United States,3.921053e+06,3.845633e+06,5.875375e+06,6.629117e+06,EB10,Creative,MA,EB10,Creative,AMER,North America
1247,Net ACV,EB10 - Creative,Iberica,2.203682e+04,4.931717e+04,7.832346e+04,4.768812e+04,EB10,Creative,MA,EB10,Creative,EMEA,Southwest Europe
2026,Net ACV,EB15 - Document Cloud,Canada,-3.669679e+04,-8.506012e+02,-4.992328e+04,-1.935086e+04,EB15,Document Cloud,MA,EB15,Document Cloud,AMER,North America
1240,Net ACV,EB10 - Creative,Benelux,7.636001e+05,2.319541e+05,2.426162e+05,7.630688e+05,EB10,Creative,MA,EB10,Creative,EMEA,Southwest Europe
1112,Net ACV,EB10 - Creative,Canada,0.000000e+00,0.000000e+00,-1.162191e+05,-1.305358e+04,EB10,Creative,MA,EB10,Creative,AMER,North America
2028,Net ACV,EB15 - Document Cloud,Canada,2.048059e+05,2.230828e+05,3.255532e+05,4.930509e+05,EB15,Document Cloud,MA,EB15,Document Cloud,AMER,North America
2048,Net ACV,EB15 - Document Cloud,Strat. Latin America,0.000000e+00,0.000000e+00,-2.246623e+01,0.000000e+00,EB15,Document Cloud,MA,EB15,Document Cloud,AMER,Latin America
2124,Net ACV,EB15 - Document Cloud,Switzerland,0.000000e+00,1.942695e+02,3.872107e+04,1.439838e+04,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Central Europe
2137,Net ACV,EB15 - Document Cloud,SSA & Israel,-2.242812e+04,2.644450e+05,5.911566e+04,2.293577e+04,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Northern Europe
2139,Net ACV,EB15 - Document Cloud,SSA & Israel,1.201843e+03,-6.038256e+03,0.000000e+00,-1.827784e+03,EB15,Document Cloud,MA,EB15,Document Cloud,EMEA,Northern Europe


In [38]:
df_DME = df_DME.drop(columns=['profit_center', 'in_parens' ])

In [39]:
df_DME.head()

,metrics,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,pc_ID,pc_descr,geo,region
1110,Net ACV,Canada,609548.258713,623444.413051,918945.896499,984383.352738,EB10,Creative,EB10,Creative,AMER,North America
1111,Net ACV,Canada,-37140.794047,-49386.365826,-116295.456059,-35329.061308,EB10,Creative,EB10,Creative,AMER,North America
1112,Net ACV,Canada,0.000000,0.000000,-116219.133330,-13053.575018,EB10,Creative,EB10,Creative,AMER,North America
1113,Net ACV,Canada,335711.243074,350931.151868,514795.103694,552220.273334,EB10,Creative,EB10,Creative,AMER,North America
1114,Net ACV,Canada,-107311.016392,-52618.206674,-43329.145301,-18793.361067,EB10,Creative,EB10,Creative,AMER,North America


In [40]:
df_DME.columns

Index(['metrics', 'market_area', 'Q1_2021', 'Q2_2021', 'Q3_2021', 'Q4_2021',
       'BU_id', 'BU_segment', 'pc_ID', 'pc_descr', 'geo', 'region'],
      dtype='object')

In [41]:
df_DME['metrics'].value_counts()

Net ACV    177
Name: metrics, dtype: int64

## To keep the columns the same as the old bookings file, we need to change some column names

- pc_descr to segment
- Add 'BU' which is all 'Digital Media'


In [42]:
df_DME.rename(columns = {'pc_descr': 'segment'}, inplace=True)

In [43]:
df_DME.sample(4)

,metrics,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021,BU_id,BU_segment,pc_ID,segment,geo,region
2120,Net ACV,Germany,1.108919e+06,1.985638e+06,2.090191e+06,4.709787e+06,EB15,Document Cloud,EB15,Document Cloud,EMEA,Central Europe
1235,Net ACV,Middle East,1.222302e+04,9.484885e+03,2.189251e+05,5.630048e+03,EB10,Creative,EB10,Creative,EMEA,Northern Europe
1131,Net ACV,Mexico,-1.685053e+04,-1.074323e+04,0.000000e+00,-9.129548e+03,EB10,Creative,EB10,Creative,AMER,Latin America
2157,Net ACV,Italy,1.589565e+04,1.417154e+04,3.040542e+04,3.665786e+04,EB15,Document Cloud,EB15,Document Cloud,EMEA,Southwest Europe


In [44]:
df_DME['BU'] = 'Digital Media'

In [45]:
# We need to remove the segment data: it is not included in the DME bookings
df_DME = df_DME.groupby(by = ['BU', 'segment', 'geo', 'region', 'market_area']).sum()
df_DME = df_DME.reset_index()

In [46]:
df_DME = df_DME[['BU', 'segment', 'geo', 'region', 'market_area', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021']]

In [47]:
df_DME.sample(10)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
11,Digital Media,Creative,EMEA,Central Europe,Eastern Europe,2.132654e+05,2.334410e+05,2.157924e+05,2.780309e+05
25,Digital Media,Document Cloud,AMER,Latin America,Mexico,-5.878793e+02,0.000000e+00,0.000000e+00,0.000000e+00
26,Digital Media,Document Cloud,AMER,Latin America,Strat. Latin America,-4.793599e+03,-1.201923e+03,-2.246623e+01,0.000000e+00
41,Digital Media,Document Cloud,EMEA,Northern Europe,SSA & Israel,-2.122628e+04,2.455625e+05,5.911566e+04,2.110799e+04
23,Digital Media,Creative,JPN,Japan,Japan,3.833277e+06,7.528496e+06,4.192102e+06,5.252535e+06
28,Digital Media,Document Cloud,AMER,North America,United States,1.941775e+07,2.345672e+07,3.555828e+07,4.887793e+07
29,Digital Media,Document Cloud,ASIA,ANZ,Aus and New Zealand,1.438565e+06,1.632384e+06,1.945140e+06,2.228751e+06
42,Digital Media,Document Cloud,EMEA,Northern Europe,United Kingdom,2.942191e+06,3.477646e+06,3.161506e+06,6.194697e+06
38,Digital Media,Document Cloud,EMEA,Central Europe,Switzerland,3.312188e+05,7.165580e+05,6.317871e+05,1.582082e+06
22,Digital Media,Creative,EMEA,Southwest Europe,Italy,1.382590e+05,1.268642e+05,3.661795e+05,3.914666e+05


In [48]:
df_DME.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BU           48 non-null     object 
 1   segment      48 non-null     object 
 2   geo          48 non-null     object 
 3   region       48 non-null     object 
 4   market_area  48 non-null     object 
 5   Q1_2021      48 non-null     float64
 6   Q2_2021      48 non-null     float64
 7   Q3_2021      48 non-null     float64
 8   Q4_2021      48 non-null     float64
dtypes: float64(4), object(5)
memory usage: 3.5+ KB


In [49]:
df_DME.sum()

BU             Digital MediaDigital MediaDigital MediaDigital...
segment         Creative Creative Creative Creative Creative ...
geo            AMERAMERAMERAMERAMERASIAASIAASIAASIAASIAASIAEM...
region         Latin AmericaLatin AmericaLatin AmericaNorth A...
market_area    BrazilMexicoStrat. Latin AmericaCanadaUnited S...
Q1_2021                                               8.3782e+07
Q2_2021                                              1.00621e+08
Q3_2021                                              1.24743e+08
Q4_2021                                               1.6877e+08
dtype: object

# Done with DME now moving to DX



In [50]:
filename = r'/Volumes/Treasury/Financial_Database/Deferred_Revenue/Inputs/DATA_2020_p12/DX_Bookings_FY21_Plan.xlsx'
sheetname = 'Sheet1'
start=12
df_DX = pd.read_excel(filename, sheetname, skiprows=start)

In [51]:
df_DX.sample(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Q1 2021,Q2 2021,Q3 2021,Q4 2021
1435,Territory,Central Europe (R),14300 - Adobe Campaign,5.201008e+05,83216.122477,109221.160751,119623.176061,2.080403e+05
1543,Territory,SSA & Israel (MA),12640 - Marketo,4.004159e+04,6006.238586,9609.981737,9009.357878,1.541601e+04
791,Strategic,Japan (R),12500 - Adobe Target,1.161500e+05,14405.000000,25785.000000,25785.000000,5.017500e+04
1656,Greenfield,All Market Areas,13030 - Forms,3.003011e+06,543463.245813,687736.451812,708046.330915,1.063765e+06
160,Enterprise,India (R),12400 - Adobe Analytics,2.328244e+06,419083.961040,512213.730160,651908.383840,7.450382e+05
88,Enterprise,Brazil (MA),13030 - Forms,2.300000e+05,42997.612011,30160.054588,63724.400566,9.311793e+04
2038,Public Sector,North America (R),12630 - Magento,1.100000e+06,146966.957266,193216.089459,317739.354421,4.420776e+05
592,Strategic,Aus and New Zealand (MA),12630 - Magento,6.932754e+05,97058.555342,138655.079060,173318.848825,2.842429e+05
1472,Territory,Eastern Europe (MA),13030 - Forms,9.066706e+04,14506.730366,19040.083606,20853.424902,3.626683e+04
874,Corporate,Latin America (R),13010 - Assets,1.375208e+06,257089.900944,180332.001798,381018.830191,5.567677e+05


In [52]:
df_DX = df_DX.rename(columns = {'Unnamed: 0': 'segment',
                                'Unnamed: 1': 'market_area',
                                'Unnamed: 2': 'profit_center',
                                'Q1 2021':'Q1_2021',
                                'Q2 2021':'Q2_2021',
                                'Q3 2021':'Q3_2021',
                                'Q4 2021':'Q4_2021' 
                                })

In [53]:
df_DX = df_DX.drop(columns = ['segment', '2021'])

## It looks like there are three seperate cuts of the data that are a hierarchy again, but they also have an 'All Market Areas' that is a total by profit center. We don't need this but there is no need to change the code here.
It will be removed when we get to the filter on region

1. G is GEO
2. R is region
3. MA is market Area

Going to split these into seperate sections and do successive fill forwards to cover all of them



In [54]:
df_DX.sample(10)

,market_area,profit_center,Q1_2021,Q2_2021,Q3_2021,Q4_2021
1303,ASIA (G),12500 - Adobe Target,157636.517167,219126.608156,263516.662822,389594.378982
1223,Japan (R),12610 - Audience Manager,9656.000000,17283.000000,17283.000000,33751.000000
667,Switzerland (MA),14800 - Customer Journey Analytics,15460.952273,20292.499859,22225.118893,38652.380683
48,Canada (MA),14600 - Adobe Exp Platform,269207.037111,378801.586933,363132.716556,443858.659400
1074,Eastern Europe (MA),12500 - Adobe Target,2654.334002,3483.813378,3815.605128,6635.835005
830,AMER (G),12610 - Audience Manager,238327.646399,167171.489044,353212.322615,516135.123146
361,Middle East (MA),13030 - Forms,15000.000000,24000.000000,22500.000000,38500.000000
87,Brazil (MA),13020 - Sites,409500.570076,287238.266723,606898.316893,886836.379907
918,ANZ (R),12640 - Marketo,26013.037393,37161.481990,46451.852487,76181.038079
1371,India (MA),13030 - Forms,40894.763657,50014.353059,64680.169310,76676.444918


In [55]:
# identify the characters in a string
df_DX['in_parens'] =  df_DX['market_area'].apply(lambda st: st[st.find("(")+1:st.find(")")])

In [56]:
df_DX['market_area'] = df_DX['market_area'].apply(lambda st: st[0:st.find("(")-1])

In [57]:
df_DX['pc_ID'] = df_DX['profit_center'].apply(lambda st: st[0:st.find('-')])
df_DX['pc_descr'] = df_DX['profit_center'].apply(lambda st: st[st.find('-')+1:])

In [58]:
df_DX['geo'] = df_DX[df_DX['in_parens']=='G']['market_area']
df_DX['geo'] =df_DX['geo'].ffill()

In [59]:
df_DX['region'] = df_DX[df_DX['in_parens']=='R']['market_area']
df_DX['region'] = df_DX['region'].ffill()

In [60]:
# filter to just include market area
df_DX = df_DX[df_DX['in_parens']=='MA'].copy()

In [61]:
# need to add the BU and segment information to the bookings file so that it matches
df_DX['BU'] = 'Digital Experience'
df_DX['segment'] = 'Experience Cloud'

In [62]:
# drop unnecessary columns and reorder the columns
df_DX = df_DX[['BU', 'segment', 'geo', 'region', 'market_area', 'Q1_2021','Q2_2021', 'Q3_2021', 'Q4_2021']]

In [63]:
# We need to remove the segment data: it is not included in the DME bookings
df_DX = df_DX.groupby(by = ['BU', 'segment', 'geo', 'region', 'market_area']).sum()
df_DX = df_DX.reset_index()


In [64]:
df_DX.head(20)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
0,Digital Experience,Experience Cloud,AMER,Latin America,Brazil,3.427051e+06,2.403855e+06,5.079044e+06,7.421805e+06
1,Digital Experience,Experience Cloud,AMER,North America,Canada,7.924589e+06,1.115070e+07,1.068946e+07,1.306577e+07
2,Digital Experience,Experience Cloud,AMER,North America,United States,1.874628e+08,2.139396e+08,2.522278e+08,3.293816e+08
3,Digital Experience,Experience Cloud,ASIA,ANZ,Aus and New Zealand,1.055118e+07,1.507311e+07,1.884138e+07,3.089987e+07
4,Digital Experience,Experience Cloud,ASIA,Greater China,China,1.210874e+06,1.816312e+06,1.967671e+06,2.573108e+06
5,Digital Experience,Experience Cloud,ASIA,Greater China,Hong Kong & Taiwan,1.348579e+06,2.022851e+06,2.191645e+06,2.866115e+06
6,Digital Experience,Experience Cloud,ASIA,India,India,4.697714e+06,5.741650e+06,7.307555e+06,8.351491e+06
7,Digital Experience,Experience Cloud,ASIA,Korea,Korea,9.818591e+05,1.472789e+06,1.595521e+06,2.086450e+06
8,Digital Experience,Experience Cloud,ASIA,SEA,Southeast Asia,3.797880e+06,5.649756e+06,5.495533e+06,7.397304e+06
9,Digital Experience,Experience Cloud,EMEA,Central Europe,Eastern Europe,8.354360e+05,1.096510e+06,1.200939e+06,2.088590e+06


In [65]:
df_DX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BU           23 non-null     object 
 1   segment      23 non-null     object 
 2   geo          23 non-null     object 
 3   region       23 non-null     object 
 4   market_area  23 non-null     object 
 5   Q1_2021      23 non-null     float64
 6   Q2_2021      23 non-null     float64
 7   Q3_2021      23 non-null     float64
 8   Q4_2021      23 non-null     float64
dtypes: float64(4), object(5)
memory usage: 1.7+ KB


# Final Check on the data


In [66]:
# DX
df_DX.head(20)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
0,Digital Experience,Experience Cloud,AMER,Latin America,Brazil,3.427051e+06,2.403855e+06,5.079044e+06,7.421805e+06
1,Digital Experience,Experience Cloud,AMER,North America,Canada,7.924589e+06,1.115070e+07,1.068946e+07,1.306577e+07
2,Digital Experience,Experience Cloud,AMER,North America,United States,1.874628e+08,2.139396e+08,2.522278e+08,3.293816e+08
3,Digital Experience,Experience Cloud,ASIA,ANZ,Aus and New Zealand,1.055118e+07,1.507311e+07,1.884138e+07,3.089987e+07
4,Digital Experience,Experience Cloud,ASIA,Greater China,China,1.210874e+06,1.816312e+06,1.967671e+06,2.573108e+06
5,Digital Experience,Experience Cloud,ASIA,Greater China,Hong Kong & Taiwan,1.348579e+06,2.022851e+06,2.191645e+06,2.866115e+06
6,Digital Experience,Experience Cloud,ASIA,India,India,4.697714e+06,5.741650e+06,7.307555e+06,8.351491e+06
7,Digital Experience,Experience Cloud,ASIA,Korea,Korea,9.818591e+05,1.472789e+06,1.595521e+06,2.086450e+06
8,Digital Experience,Experience Cloud,ASIA,SEA,Southeast Asia,3.797880e+06,5.649756e+06,5.495533e+06,7.397304e+06
9,Digital Experience,Experience Cloud,EMEA,Central Europe,Eastern Europe,8.354360e+05,1.096510e+06,1.200939e+06,2.088590e+06


In [67]:
# DME
df_DME.sample(20)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
37,Digital Media,Document Cloud,EMEA,Central Europe,Russia & CIS,3.529556e+04,4.880611e+04,5.010099e+04,1.160239e+05
18,Digital Media,Creative,EMEA,Northern Europe,United Kingdom,3.448298e+06,3.228927e+06,3.991542e+06,6.352543e+06
17,Digital Media,Creative,EMEA,Northern Europe,SSA & Israel,1.205584e+05,-4.262344e+04,2.048744e+04,1.364170e+05
0,Digital Media,Creative,AMER,Latin America,Brazil,1.151616e+06,1.118700e+06,1.765603e+06,1.978504e+06
23,Digital Media,Creative,JPN,Japan,Japan,3.833277e+06,7.528496e+06,4.192102e+06,5.252535e+06
21,Digital Media,Creative,EMEA,Southwest Europe,Iberica,1.394746e+05,3.527719e+05,7.197105e+05,4.926122e+05
47,Digital Media,Document Cloud,JPN,Japan,Japan,1.677403e+06,2.320611e+06,2.051876e+06,2.582073e+06
43,Digital Media,Document Cloud,EMEA,Southwest Europe,Benelux,6.114940e+05,7.775683e+05,1.139297e+06,1.174780e+06
39,Digital Media,Document Cloud,EMEA,Northern Europe,Middle East,5.293160e+04,1.574121e+05,2.337120e+05,2.994339e+05
32,Digital Media,Document Cloud,ASIA,India,India,7.569294e+05,8.381681e+05,9.769015e+05,1.044341e+06


In [68]:
df_DME.sample(5)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
18,Digital Media,Creative,EMEA,Northern Europe,United Kingdom,3.448298e+06,3.228927e+06,3.991542e+06,6.352543e+06
26,Digital Media,Document Cloud,AMER,Latin America,Strat. Latin America,-4.793599e+03,-1.201923e+03,-2.246623e+01,0.000000e+00
45,Digital Media,Document Cloud,EMEA,Southwest Europe,Iberica,1.911192e+05,3.210437e+05,5.200817e+05,4.827698e+05
2,Digital Media,Creative,AMER,Latin America,Strat. Latin America,-3.929125e+04,-1.180557e+04,-1.417209e+04,-1.025229e+04
33,Digital Media,Document Cloud,ASIA,Korea,Korea,1.840781e+05,3.152377e+05,3.837583e+05,4.280308e+05


In [69]:
df_DME.columns

Index(['BU', 'segment', 'geo', 'region', 'market_area', 'Q1_2021', 'Q2_2021',
       'Q3_2021', 'Q4_2021'],
      dtype='object')

In [70]:
df_DX.sample(5)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
18,Digital Experience,Experience Cloud,EMEA,Southwest Europe,Benelux,4.788053e+06,6.650073e+06,5.054056e+06,1.010811e+07
1,Digital Experience,Experience Cloud,AMER,North America,Canada,7.924589e+06,1.115070e+07,1.068946e+07,1.306577e+07
19,Digital Experience,Experience Cloud,EMEA,Southwest Europe,France,7.019966e+06,9.749952e+06,7.409964e+06,1.481993e+07
0,Digital Experience,Experience Cloud,AMER,Latin America,Brazil,3.427051e+06,2.403855e+06,5.079044e+06,7.421805e+06
12,Digital Experience,Experience Cloud,EMEA,Central Europe,Switzerland,3.382828e+06,4.439961e+06,4.862815e+06,8.457069e+06


# Checking totals


In [71]:
df_DX.sum()

BU             Digital ExperienceDigital ExperienceDigital Ex...
segment        Experience CloudExperience CloudExperience Clo...
geo            AMERAMERAMERASIAASIAASIAASIAASIAASIAEMEAEMEAEM...
region         Latin AmericaNorth AmericaNorth AmericaANZGrea...
market_area    BrazilCanadaUnited StatesAus and New ZealandCh...
Q1_2021                                              2.88639e+08
Q2_2021                                              3.58238e+08
Q3_2021                                              3.98039e+08
Q4_2021                                              5.68557e+08
dtype: object

In [72]:
df_DME.sum()

BU             Digital MediaDigital MediaDigital MediaDigital...
segment         Creative Creative Creative Creative Creative ...
geo            AMERAMERAMERAMERAMERASIAASIAASIAASIAASIAASIAEM...
region         Latin AmericaLatin AmericaLatin AmericaNorth A...
market_area    BrazilMexicoStrat. Latin AmericaCanadaUnited S...
Q1_2021                                               8.3782e+07
Q2_2021                                              1.00621e+08
Q3_2021                                              1.24743e+08
Q4_2021                                               1.6877e+08
dtype: object

In [73]:
df = pd.concat([df_DME, df_DX])

In [75]:
df.sample(10)

,BU,segment,geo,region,market_area,Q1_2021,Q2_2021,Q3_2021,Q4_2021
7,Digital Media,Creative,ASIA,Greater China,Hong Kong & Taiwan,5.611928e+05,6.349809e+05,7.085147e+05,7.295253e+05
45,Digital Media,Document Cloud,EMEA,Southwest Europe,Iberica,1.911192e+05,3.210437e+05,5.200817e+05,4.827698e+05
30,Digital Media,Document Cloud,ASIA,Greater China,China,1.085210e+06,1.131169e+06,1.127046e+06,1.329854e+06
13,Digital Experience,Experience Cloud,EMEA,Med,Mediterranean,-1.920000e+06,-2.800000e+06,-2.800000e+06,-3.880000e+06
40,Digital Media,Document Cloud,EMEA,Northern Europe,Nordic,9.552017e+05,1.216563e+06,9.508796e+05,2.963517e+06
16,Digital Experience,Experience Cloud,EMEA,Northern Europe,SSA & Israel,4.536430e+05,7.258289e+05,6.804646e+05,1.164350e+06
1,Digital Experience,Experience Cloud,AMER,North America,Canada,7.924589e+06,1.115070e+07,1.068946e+07,1.306577e+07
0,Digital Media,Creative,AMER,Latin America,Brazil,1.151616e+06,1.118700e+06,1.765603e+06,1.978504e+06
42,Digital Media,Document Cloud,EMEA,Northern Europe,United Kingdom,2.942191e+06,3.477646e+06,3.161506e+06,6.194697e+06
20,Digital Experience,Experience Cloud,EMEA,Southwest Europe,Iberica,2.889054e+06,4.012575e+06,3.049557e+06,6.099113e+06


In [76]:
df_test = pd.melt(df, id_vars = ['BU', 'segment', 'geo', 'region', 'market_area'], 
                 value_vars = ['Q1_2021', 'Q2_2021', 'Q3_2021', 'Q4_2021'], 
                 var_name = 'Quarter')

In [78]:
df_test.sample(10)

,BU,segment,geo,region,market_area,Quarter,value
2,Digital Media,Creative,AMER,Latin America,Strat. Latin America,Q1_2021,-3.929125e+04
75,Digital Media,Creative,AMER,North America,United States,Q2_2021,1.985063e+07
137,Digital Experience,Experience Cloud,EMEA,Southwest Europe,Benelux,Q2_2021,6.650073e+06
214,Digital Media,Creative,AMER,Latin America,Mexico,Q4_2021,-9.129548e+03
242,Digital Media,Document Cloud,ASIA,ANZ,Aus and New Zealand,Q4_2021,2.228751e+06
176,Digital Media,Document Cloud,ASIA,SEA,Southeast Asia,Q3_2021,8.999795e+05
18,Digital Media,Creative,EMEA,Northern Europe,United Kingdom,Q1_2021,3.448298e+06
213,Digital Media,Creative,AMER,Latin America,Brazil,Q4_2021,1.978504e+06
54,Digital Experience,Experience Cloud,ASIA,India,India,Q1_2021,4.697714e+06
10,Digital Media,Creative,ASIA,SEA,Southeast Asia,Q1_2021,1.137801e+06


In [79]:
list_ma = df_test['market_area'].unique()

In [80]:
list_ma


array(['Brazil', 'Mexico', 'Strat. Latin America', 'Canada',
       'United States', 'Aus and New Zealand', 'China',
       'Hong Kong & Taiwan', 'India', 'Korea', 'Southeast Asia',
       'Eastern Europe', 'Germany', 'Russia & CIS', 'Switzerland',
       'Middle East', 'Nordic', 'SSA & Israel', 'United Kingdom',
       'Benelux', 'France', 'Iberica', 'Italy', 'Japan', 'Mediterranean'],
      dtype=object)